## Pyhton и подключение к БД

*Для выполнения задания Вам представлен датасет с перелётами пассажиров, а также шаблон Jupyter Notebook с переченем задач, которые необходимо выполнить, используя новые навыки, полученные в данной теме. Результат каждого задания должен быть получен посредством выполнения SQL-запроса*


### Импорт библиотек

In [12]:
import pandas as pd
import psycopg2
from dotenv import load_dotenv

import os
import warnings


### Глобальные настройки

In [13]:
warnings.filterwarnings("ignore", category=UserWarning)

load_dotenv()

True

### Подключение к базе

In [14]:
conn = psycopg2.connect(host=os.getenv('DB_HOST'),
                        port=os.getenv('DB_PORT'),
                        dbname=os.getenv('DB_NAME'),
                        user=os.getenv('DB_USER'),
                        password=os.getenv('DB_PASSWORD'),
                        options="-c search_path=AERO,public")


### Знакомство с БД

In [15]:
cur = conn.cursor()
cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='AERO'")
shqmas_tables = [i[0] for i in cur.fetchall()]

df_trip = pd.read_sql('SELECT * FROM "AERO"."trip"', conn)
df_company = pd.read_sql('SELECT * FROM "AERO"."company"', conn)
df_pass_in_trip = pd.read_sql('SELECT * FROM "AERO"."pass_in_trip"', conn)
df_passenger = pd.read_sql('SELECT * FROM "AERO"."passenger"', conn)

print(f"Таблицы: {', '.join(shqmas_tables)}")

print('trip', end=': ')
display(df_trip.head(3))
print('company', end=': ')
display(df_company.head(3))
print('pass_in_trip', end=': ')
display(df_pass_in_trip.head(3))
print('passenger', end=': ')
display(df_passenger.head(3))

Таблицы: trip, company, passenger, pass_in_trip
trip: 

,trip_no,id_comp,plane,town_from,town_to,time_out,time_in
0,1100,4,Boeing,Rostov,Paris,1900-01-01 14:30:00,1900-01-01 17:50:00
1,1101,4,Boeing,Paris,Rostov,1900-01-01 08:12:00,1900-01-01 11:45:00
2,1123,3,TU-154,Rostov,Vladivostok,1900-01-01 16:20:00,1900-01-01 03:40:00


company: 

,id_comp,name
0,1,Don_avia
1,2,Aeroflot
2,3,Dale_avia


pass_in_trip: 

,trip_no,date,id_psg,place
0,1100,2003-04-29,1,1a
1,1123,2003-04-05,3,2a
2,1123,2003-04-08,1,4c


passenger: 

,id_psg,name
0,1,Bruce Willis
1,2,George Clooney
2,3,Kevin Costner


1. Вывести имена всех людей, которые есть в базе данных авиакомпаний

In [16]:
passenger = pd.read_sql(
    '''
    SELECT name
      FROM "AERO"."passenger";
    '''
    , conn)

display(passenger)

,name
0,Bruce Willis
1,George Clooney
2,Kevin Costner
3,Donald Sutherland
4,Jennifer Lopez
5,Ray Liotta
6,Samuel L. Jackson
7,Nikole Kidman
8,Alan Rickman
9,Kurt Russell


2. Вывести названия всеx авиакомпаний

In [17]:
name_company = pd.read_sql(
    '''
    SELECT name
      FROM "AERO"."company";
    '''
    , conn)

display(name_company)

,name
0,Don_avia
1,Aeroflot
2,Dale_avia
3,air_France
4,British_AW


3. Вывести все рейсы, совершенные из Москвы

In [18]:
trip_from_moscow = pd.read_sql(
    """
    SELECT trip_no
      FROM "AERO"."trip"
     WHERE town_from = 'Moscow';
    """
    , conn)

display(trip_from_moscow)

,trip_no
0,1145
1,1182
2,1188
3,1196


4. Вывести имена людей, которые заканчиваются на "man"

In [19]:
passenger_name_man = pd.read_sql(
    """
    SELECT name
      FROM "AERO"."passenger"
     WHERE name LIKE '%man%';
    """
    , conn)

display(passenger_name_man)

,name
0,Nikole Kidman
1,Alan Rickman
2,Gary Oldman


5. Вывести количество рейсов, совершенных на TU-134

In [20]:
count_trip_tu_134 = pd.read_sql(
    """
    SELECT plane,
           COUNT(*)
      FROM "AERO"."trip"
     GROUP BY plane
    HAVING plane = 'TU-134';
    """
    , conn)

display(count_trip_tu_134)

,plane,count
0,TU-134,4


6. Какие компании совершали перелеты на Boeing

In [21]:
company_on_boing = pd.read_sql(
    """
    SELECT DISTINCT company.name,
                    trip.plane
      FROM "AERO"."trip" AS trip
           JOIN "AERO"."company" AS company USING(id_comp)
     WHERE trip.plane = 'Boeing';
    """
    , conn)

display(company_on_boing)

,name,plane
0,air_France,Boeing
1,British_AW,Boeing


7. Вывести все названия самолётов, на которых можно улететь в Москву (Moscow)

In [22]:
plane_to_moscow = pd.read_sql(
    """
    SELECT DISTINCT plane
      FROM "AERO"."trip"
     WHERE town_to = 'Moscow';
    """
    , conn)

display(plane_to_moscow)

,plane
0,IL-86
1,TU-134
2,TU-154


8. В какие города можно улететь из Парижа (Paris) и сколько времени это займёт?

In [23]:
paris_to_city = pd.read_sql(
    """
    SELECT town_from,
           town_to,
           time_in - time_out AS travel_time
      FROM "AERO"."trip"
     WHERE town_from = 'Paris';
    """
    , conn)

display(paris_to_city)

,town_from,town_to,travel_time
0,Paris,Rostov,0 days 03:33:00
1,Paris,London,0 days 01:00:00


9. Какие компании организуют перелеты из Владивостока (Vladivostok)?

In [24]:
company_on_vladivostok = pd.read_sql(
    """
    SELECT company.name
      FROM "AERO"."trip" AS trip
           JOIN "AERO"."company" AS company USING(id_comp)
     WHERE trip.town_from = 'Vladivostok';
    """
    , conn)

display(company_on_vladivostok)

,name
0,Dale_avia


10. Вывести вылеты, совершенные с 10 ч. по 14 ч. 1 января 1900 г.

In [25]:
flight_spec_time = pd.read_sql(
    """
    SELECT *
      FROM "AERO"."trip"
     WHERE EXTRACT(YEAR FROM time_out) = 1900 AND
           EXTRACT(MONTH FROM time_out) = 1 AND
           EXTRACT(DAY FROM time_out) = 1 AND
           EXTRACT(HOUR FROM time_out) BETWEEN 10 AND 14;
    """
    , conn)

display(flight_spec_time)

,trip_no,id_comp,plane,town_from,town_to,time_out,time_in
0,1100,4,Boeing,Rostov,Paris,1900-01-01 14:30:00,1900-01-01 17:50:00
1,1182,1,TU-134,Moscow,Rostov,1900-01-01 12:35:00,1900-01-01 14:30:00
2,7772,5,Boeing,Singapore,London,1900-01-01 12:00:00,1900-01-01 02:00:00
3,7774,5,Boeing,Singapore,London,1900-01-01 14:00:00,1900-01-01 06:00:00


11. Вывести пассажиров с самым длинным именем

In [26]:
pasenger_len_name = pd.read_sql(
    """
    WITH name_len AS(
        SELECT name,
              LENGTH(name) AS lenght_name,
              MAX(LENGTH(name)) OVER () AS max_length
          FROM "AERO"."passenger"
        ORDER BY lenght_name DESC
    )

    SELECT *
      FROM name_len
     WHERE lenght_name = max_length;
    """
    , conn)

display(pasenger_len_name)

,name,lenght_name,max_length
0,Catherine Zeta-Jones,20,20


12. Вывести id и количество пассажиров для всех прошедших полётов

In [27]:
# Не понятное задание , понял его так

trip_pass_id = pd.read_sql(
    """
    WITH count_psg AS(
    SELECT trip.trip_no,
           pass_in_trip.id_psg,
           passenger.name
      FROM "AERO"."trip" AS trip
           JOIN "AERO"."pass_in_trip" USING(trip_no)
           JOIN "AERO"."passenger" USING(id_psg)
    )

    SELECT trip_no,
           COUNT(id_psg) AS count_passenger
      FROM count_psg
     GROUP BY trip_no;
    """
    , conn)

display(trip_pass_id)

,trip_no,count_passenger
0,1145,2
1,1187,2
2,1123,3
3,7771,6
4,1124,1
5,8882,2
6,1100,1
7,8881,1
8,7773,1
9,1181,4


13. Вывести имена людей, у которых есть полный тёзка среди пассажиров

In [28]:
namesake = pd.read_sql(
    """
    SELECT name
      FROM "AERO"."passenger"
     GROUP BY name
    HAVING COUNT(*) > 1;
    """
    , conn)

display(namesake)

,name
0,Bruce Willis


14. В какие города летал Bruce Willis

In [29]:
city_bruce = pd.read_sql(
    """
    SELECT passenger.name,
           trip.town_to
      FROM "AERO"."trip" AS trip
           JOIN "AERO"."pass_in_trip" USING(trip_no)
           JOIN "AERO"."passenger" USING(id_psg)
     WHERE name = 'Bruce Willis';

    """
    , conn)

display(city_bruce)

,name,town_to
0,Bruce Willis,Paris
1,Bruce Willis,Vladivostok
2,Bruce Willis,Moscow


15. Выведите дату и время прилёта пассажира Стив Мартин (Steve Martin) в Лондон (London)

In [30]:
date_from_to_martin = pd.read_sql(
    """
    SELECT passenger.name,
           trip.town_to,
           trip.time_in
      FROM "AERO"."trip" AS trip
           JOIN "AERO"."pass_in_trip" USING(trip_no)
           JOIN "AERO"."passenger" USING(id_psg)
     WHERE name = 'Steve Martin' AND town_to = 'London';

    """
    , conn)

display(date_from_to_martin)

,name,town_to,time_in
0,Steve Martin,London,1900-01-01 02:00:00


16. Вывести отсортированный по количеству перелетов (по убыванию) и имени (по возрастанию) список пассажиров, совершивших хотя бы 1 полет.

In [31]:
sort_trip = pd.read_sql(
    """
    SELECT passenger.name,
           COUNT(pass_in_trip.place) AS count_place
      FROM "AERO"."passenger" AS passenger
           JOIN "AERO"."pass_in_trip" USING(id_psg)
     GROUP BY passenger.name
     ORDER BY count_place DESC, name;
    """
    , conn)

display(sort_trip)

,name,count_place
0,Michael Caine,4
1,Mullah Omar,4
2,Bruce Willis,3
3,Harrison Ford,3
4,Jennifer Lopez,3
5,Kurt Russell,3
6,Nikole Kidman,3
7,Kevin Costner,2
8,Ray Liotta,2
9,Steve Martin,2


### Закрываем соединение

In [32]:
conn.close()